In [30]:
import logging

# Настройки логгера
logging.basicConfig(level=logging.INFO, filename="py_log.log", filemode="w", encoding="utf-8")

In [31]:
import time
import random
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys

In [ ]:
# Настройки Selenium
chrome_options = Options()
chrome_options.add_argument("--headless")  # Запуск без GUI
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")

driver = webdriver.Chrome()

BASE_URLS = [
    "https://ria.ru/politics/",
    "https://ria.ru/world/",
    "https://ria.ru/economy/",
    "https://ria.ru/society/",
    "https://ria.ru/incidents/",
    "https://ria.ru/defense_safety/"
]

# Функция для получения ссылок на новости
def get_news(max_pages):
    for BASE_URL in BASE_URLS:
        logging.info(msg=f"Парсинг {BASE_URL.split("/")[-2]}")
        news = []
        driver.get(BASE_URL)
        time.sleep(random.random()+2)

        for _ in range(max_pages):
            # Прокрутка страницы вниз для подгрузки контента
            driver.find_element(By.TAG_NAME, "body").send_keys(Keys.END)
            time.sleep(3)
        
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        for item in soup.find_all('div', {"class": "list-item"}):
            current = {}

            current['text'] = item.find("div", {"class": "list-item__content"}).find("a", {"class": "list-item__title"}).text

            for info in item.find("div", {"class": "list-item__info"}).find_all("div", {"class": "list-item__info-item"}):
                if info.find("span") is None:
                    current['date'] = info.text
                else:
                    current['views'] = info.find("span").text

            current['tags'] = []
            for tag in item.find("div", {"class": "list-item__tags"}).find("div", {"class": "list-item__tags-list"}).find_all("a", {"class": "list-tag"}):
                current['tags'].append(tag.find("span", {"class": "list-tag__text"}).text)
                
            news.append(current)

        news_df = pd.DataFrame(news)
        news_df.to_csv(f"{BASE_URL.split("/")[-2]}.csv", index=False, encoding="utf-8-sig")
        logging.info(msg=f"Парсинг {BASE_URL.split("/")[-2]} прошёл успешно. Значений пропаршено {news_df.shape[0]}")
        
    return news

get_news(100)
driver.quit()

logging.info(msg=f"Скрепинг завершен")

In [35]:
df = pd.read_csv('politics.csv')
df.head()

,text,date,views,tags
0,Путин назначил Баланина замглавы Минюста России,14:41,1424,"['Политика', 'Россия', 'Владимир Путин', 'Всев..."
1,Избранный президент Абхазии назвал отношения с...,14:34,4921,"['Политика', 'Россия', 'Абхазия', 'В мире', 'Б..."
2,"Путин отметил мужество жителей Татарстана, вою...",13:53,1055,"['Политика', 'Россия', 'Владимир Путин', 'Руст..."
3,Хинштейн заявил о саботаже его поручений курск...,12:51,55854,"['Политика', 'Железногорск ', 'Курская область..."
4,Путин в понедельник встретится с главой Татарс...,12:39,659,"['Политика', 'Республика Татарстан (Татарстан)..."


In [36]:
for theme in BASE_URLS[1:]:
    temp = pd.read_csv(f"{theme.split("/")[-2]}.csv")
    df = pd.concat([df, temp], ignore_index=True)

In [37]:
df.to_csv("scraping.csv")